In [1]:
# Library imports
import pyforest
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
from pprint import pprint
from time import sleep
import time
import seaborn as sns

from turtle import forward
import torch.nn as nn
import torch.nn.functional as F
import math
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
from torch.utils.data import DataLoader, TensorDataset

In [2]:
!ls

combined_data_1.txt  netflix.ipynb


#### **Formatting the data**

In [3]:
fopen = open('combined_data_1.txt', 'r')
fwrite = open('data.txt', 'a+')
fopen.seek(0)
fwrite.seek(0)
fwrite.writelines('CustomerID,Score,Date,MovieID\n')

for line in fopen.readlines():
    line = line.strip()
    if line[-1] == ':':
        movie_id = line[:-1]
    else:
        fwrite.writelines(line + ',' + movie_id + '\n')

fopen.close()
fwrite.close()

#### **Exploring the data**